In [1]:
import sys
import os
import subprocess
from math import *
import numpy as np
from datetime import *
from pylab import *
import matplotlib as mpl
from matplotlib.widgets import Slider, Button, RadioButtons
import matplotlib.pyplot as plt
from astropy.table import Table, Column 
from mpl_toolkits.axes_grid1 import make_axes_locatable
from optparse import OptionParser
from PIL import Image#, ImageTk
from subprocess import Popen, PIPE
import matplotlib.patches as patches
import scipy.misc as scimisc
import pandas as pd

In [2]:
def p_append(params, email, p):
    
    n = 0
    if email=='rbtully1@gmail.com': n = 4
    elif email=='rtully@hawaii.edu': n = 4
    elif email=='ekourkchi@gmail.com': n = 3
    elif email=='s.eftekharzadeh@gmail.com':
        n=3
    elif email=='mokelkea@hawaii.edu': n = 3
    elif email=='chasemu@hawaii.edu': n = 3
    elif email=='jrl2014@hawaii.edu': 
        n=2
    elif email=='dschoen@hawaii.edu': n = 3
    elif email=='adholtha@hawaii.edu': n = 4
    elif email=='chuangj@hawaii.edu': n = 2
    elif email=='mi24@hawaii.edu': n = 3
    elif email=='mka7@hawaii.edu': n = 2
    elif email=='a.danesh61@gmail.com': n = 2
    
    elif email=='cgrubner0@gmail.com': n = 1
    elif email=='pascal.jouve@free.fr': n = 2
    elif email=='dlsaintsorny@gmail.com': n = 2
    elif email=='arnaud.ohet@gmail.com': n = 1
    elif email=='hawaii@udrea.fr': n = 2
    elif email=='helenecourtois33@gmail.com': n = 2
    elif email=='claude.rene21@gmail.com': n = 1
    elif email=='fredwallet@gmail.com': n = 1
    elif email=='henri140860@wanadoo.fr': n = 1
    elif email=='joannin.lycee@free.fr': n = 2
    elif email=='bevig434@gmail.com': n = 1
    elif email=='pierrefcevey@gmail.com': n = 1
    elif email=='pierre@macweber.ch': n = 1
    elif email=='arnaudoech@gmail.com': n = 1
    elif email=='lionmarm@gmail.com': n = 1
    elif email=='neilljd@gmail.com': n = 3
    elif email=='mseibert@carnegiescience.edu': n = 3
        
    for i in range(n): params.append(p)
        
    return params  

def median_param(param_list, param_email):
    
    p_lst = []
    N = len(param_list)
    for i in range(N):
        p_lst = p_append(p_lst, param_email[i], param_list[i])
    return np.median(p_lst)
    
    

In [3]:
inFile = 'EDD_distance_cf4_v27.csv'
table  = np.genfromtxt(inFile , delimiter='|', filling_values=-1, names=True, dtype=None)
pgc  = table['pgc']
inc  = table['inc']
face_on  = table['fon']
inc_note = table['inc_note']
inc_flg = table['inc_flg']

/home/ehsan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  


In [4]:
inFile = 'std_scales.csv'
table  = np.genfromtxt(inFile , delimiter=',', filling_values=-1, names=True, dtype=None)
pgc_std  = table['pgcID']
dPA_std  = table['dPA']
zoom_std  = table['zoom']

inFile = 'users_scales.csv'
table  = np.genfromtxt(inFile , delimiter=',', filling_values=-1, names=True, dtype=None)
pgc_user  = table['pgcID']
dPA_user  = table['dPA']
zoom_user  = table['zoom']
email_user  = [' '.join(dummy.split()) for dummy in table['email']]


/home/ehsan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  
/home/ehsan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  


In [9]:
######################################
def xcmd(cmd,verbose):

    if verbose: print '\n'+cmd

    tmp=os.popen(cmd)
    output=''
    for x in tmp: output+=x
    if 'abort' in output:
        failure=True
    else:
        failure=tmp.close()
    if False:
        print 'execution of %s failed' % cmd
        print 'error is as follows',output
        sys.exit()
    else:
        return output

######################################

def converIMAGE(im_root, pgcID, angle=0., scale=1., size=64, suffix=None):
    
    if scale<1.:
        scale=1

    img = Image.open(im_root + 'pgc'+str(pgcID)+'_d25x2_rot_gri.sdss.jpg')

    img_rot = scimisc.imrotate(img, -angle, interp='bilinear')

    img_rot = np.asarray(img_rot)

    N = img_rot.shape
    d = N[0]
    p =  int(d/scale)
    d1 = d/2-p/2
    d2 = d1 + p

    img_cut = img_rot[d1:d2, d1:d2, :]

    img = Image.fromarray(img_cut, 'RGB').resize((size,size))

    outDIR = './' + str(size)+'x'+str(size) + ''
    if not os.path.exists(outDIR):
        xcmd('mkdir '+outDIR, True)
    
    if not suffix is None:
        outName = outDIR+'/pgc'+str(pgcID)+'_'+str(size)+'x'+str(size)+'_'+suffix+'.jpg'
    else:
        outName = outDIR+'/pgc'+str(pgcID)+'_'+str(size)+'x'+str(size)+'.jpg'

    img.save(outName, "JPEG")
    
    return img

In [6]:
# im_path =  '../galaxies/'+'pgc3957878_d25x2_rot_gri.sdss.jpg'
# im = converIMAGE('../galaxies/', 3957878, angle=0, scale=0.5, size=64)



# plt.imshow(im)

# id = 3957878
# dPA = 0
# zoom = 1.0
# if id in pgc_std:
#     ix,  = np.where(pgc_std==id)
#     dPA  = dPA_std[ix][0]
#     zoom = zoom_std[ix][0]
# elif id in pgc_user:
#     ix,  = np.where(pgc_user==id)
#     emails = [email_user[i] for i in ix]
#     dPA  = median_param(dPA_user[ix], emails)
#     zoom = median_param(zoom_user[ix],emails)

# print dPA, zoom

In [8]:
im_root = './galaxies/'
for j, id in enumerate(pgc):
    
    im_path =  im_root+'pgc'+str(id)+'_d25x2_rot_gri.sdss.jpg'
    
    available = os.path.exists(im_path)
    
    fon = " ".join(face_on[j].split())
    if ((fon == 'F' and inc_flg[j]>0) or (inc_flg[j]>0 and 'face_on' in inc_note[j])):
        suffix = 'F'
    elif inc_flg[j]>0:
        suffix = '0'
    else:
        suffix = "%d"%inc[j]
    
    
    if available:
        
        dPA = 0
        zoom = 1.0
        if id in pgc_std:
            ix,  = np.where(pgc_std==id)
            dPA  = dPA_std[ix][0]
            zoom = zoom_std[ix][0]
        elif id in pgc_user:
            ix,  = np.where(pgc_user==id)
            emails = [email_user[i] for i in ix]
            dPA  = median_param(dPA_user[ix], emails)
            zoom = median_param(zoom_user[ix],emails)
        try: 
            converIMAGE(im_root, id, angle=dPA, scale=zoom, size=128, suffix=suffix)
        except:
            print 'Problem: ' + im_path

/home/ehsan/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: `imrotate` is deprecated!
`imrotate` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.rotate`` instead.



mkdir ./128x128
Problem: ../galaxies/pgc3648947_d25x2_rot_gri.sdss.jpg
Problem: ../galaxies/pgc5057040_d25x2_rot_gri.sdss.jpg
Problem: ../galaxies/pgc5057372_d25x2_rot_gri.sdss.jpg
Problem: ../galaxies/pgc5057571_d25x2_rot_gri.sdss.jpg
Problem: ../galaxies/pgc5058874_d25x2_rot_gri.sdss.jpg
Problem: ../galaxies/pgc5058906_d25x2_rot_gri.sdss.jpg
Problem: ../galaxies/pgc5059958_d25x2_rot_gri.sdss.jpg
Problem: ../galaxies/pgc5060016_d25x2_rot_gri.sdss.jpg
